In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from torch.utils.data import Dataset
import pickle
import datetime
import pandas as pd


In [ ]:
import csv
training_data = []
with open('database_paris_marseille copy.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        row[0] = float(row[0])
        if row[0]>90:
            row[0] = row[0]-360
        training_data.append([float(item) for item in row])


with open("objet_wind_data_2020.pickle", "rb") as f:
    wind_data = pickle.load(f)
    data = wind_data['data']



In [ ]:
print(wind_data['metadata'])
print(wind_data['metadata']['grid']['latitude'][55])
print(grid_long(359))

{'dataset': 'NOAA', 'grid': {'time': [datetime.datetime(2020, 1, 1, 0, 0), datetime.datetime(2020, 1, 1, 6, 0), datetime.datetime(2020, 1, 1, 12, 0), datetime.datetime(2020, 1, 1, 18, 0), datetime.datetime(2020, 1, 2, 0, 0), datetime.datetime(2020, 1, 2, 6, 0), datetime.datetime(2020, 1, 2, 12, 0), datetime.datetime(2020, 1, 2, 18, 0), datetime.datetime(2020, 1, 3, 0, 0), datetime.datetime(2020, 1, 3, 6, 0), datetime.datetime(2020, 1, 3, 12, 0), datetime.datetime(2020, 1, 3, 18, 0), datetime.datetime(2020, 1, 4, 0, 0), datetime.datetime(2020, 1, 4, 6, 0), datetime.datetime(2020, 1, 4, 12, 0), datetime.datetime(2020, 1, 4, 18, 0), datetime.datetime(2020, 1, 5, 0, 0), datetime.datetime(2020, 1, 5, 6, 0), datetime.datetime(2020, 1, 5, 12, 0), datetime.datetime(2020, 1, 5, 18, 0), datetime.datetime(2020, 1, 6, 0, 0), datetime.datetime(2020, 1, 6, 6, 0), datetime.datetime(2020, 1, 6, 12, 0), datetime.datetime(2020, 1, 6, 18, 0), datetime.datetime(2020, 1, 7, 0, 0), datetime.datetime(2020, 1

In [ ]:
def grid_time(t):
    t = t
    t = t/3600
    t = t + 1 + 4
    t = t - t%6
    t = t - 1
    return (t+1)/6

def grid_lat(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5
    l = l/2
    l = l +90
    l = l*2

    return (l/5)%73

def grid_long(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5

    return (l/5)%144
def int_array(arr):
    return [int(i) for i in arr]

def grid_array(arr):
    return int_array([grid_long(arr[0]),grid_lat(arr[1]),grid_time(arr[2]),int(arr[3]),int(arr[4])])

def get_winds_at_t(data,time,dtime):
    tensor = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    return tensor

def input_label(training_data,dtime,i, goal_coord,data):
    array = grid_array(training_data[i])
    time = array[2]
    long = array[0]
    lat = array[1]
    alt = array[3]
    action = array[4]
    #winds = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    winds = data[time:time+dtime,:,:,:]
    winds = np.transpose(winds,(0,4,1,2,3))
    pos = np.zeros((dtime,1,winds.shape[2],winds.shape[3],winds.shape[4]))
    for i in range(pos.shape[2]):
        pos[0,0,i,int(grid_long(goal_coord[0])),int(grid_lat(goal_coord[1]))] = -10
    pos[0,0,alt,long,lat] = 10
    input = np.concatenate((winds,pos),axis=1)

    return (input,action)


In [ ]:
coords_destination = [5.3698, 43.2965]
array = training_data[np.random.randint(0,len(training_data))]
print(array)
grid = grid_array(array)
print(grid)
print(wind_data['metadata']['grid']['latitude'][grid[1]])

[5.273628430259066, 45.386565605055054, 17614800.0, 15.0, 0.0]
[2, 54, 816, 15, 0]
45.0


In [ ]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(3, 16, 5, padding=(2,2,2))
        self.conv2 = nn.Conv3d(16, 32, 5, padding=(2,2,2))
        self.conv3 = nn.Conv3d(32, 64, 3, padding=(1,1,1))
        self.conv4 = nn.Conv3d(64, 128, 3, padding=(1,1,1))
        self.conv5 = nn.Conv3d(128, 256, 3, padding=(1,1,1))
        self.conv6 = nn.Conv3d(256, 256, 3, padding=(1,1,1))
        self.conv7 = nn.Conv3d(256, 384, 3, padding=(1,1,1))

        self.fc1 = nn.Linear(1152, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 3)
        self.drop = nn.Dropout(0.3)

        self.batch1 = nn.BatchNorm3d(16)
        self.batch2 = nn.BatchNorm3d(32)
        self.batch3 = nn.BatchNorm3d(64)
        self.batch4 = nn.BatchNorm3d(128)
        self.batch5 = nn.BatchNorm3d(256)
        self.batch6 = nn.BatchNorm3d(256)
        self.batch7 = nn.BatchNorm3d(384)

        self.batch8 = nn.BatchNorm1d(512)
        self.batch9 = nn.BatchNorm1d(512)
        self.batch10 = nn.BatchNorm1d(512)


        self.pool= nn.MaxPool3d(kernel_size=(1,3,3),stride = (1,2,2))
        self.pool1= nn.MaxPool3d(kernel_size=(2,2,2),stride = (2,2,2))
        self.pool2= nn.MaxPool3d(kernel_size=(1,2,1),stride = (1,2,1))

        # self.pool1 = nn.MaxPool3d(kernel_size=,stride = 2)


        n = self.conv1.kernel_size[0] * self.conv1.kernel_size[1] * self.conv1.kernel_size[2]* self.conv1.out_channels
        self.conv1.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv2.kernel_size[0] * self.conv2.kernel_size[1] * self.conv1.kernel_size[2]* self.conv2.out_channels
        self.conv2.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv3.kernel_size[0] * self.conv3.kernel_size[1] * self.conv1.kernel_size[2]* self.conv3.out_channels
        self.conv3.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv4.kernel_size[0] * self.conv4.kernel_size[1] * self.conv1.kernel_size[2]* self.conv4.out_channels
        self.conv4.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv5.kernel_size[0] * self.conv5.kernel_size[1] * self.conv1.kernel_size[2]* self.conv5.out_channels
        self.conv5.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv6.kernel_size[0] * self.conv6.kernel_size[1] * self.conv1.kernel_size[2]* self.conv6.out_channels
        self.conv6.weight.data.normal_(0, math.sqrt(2. / n))
        n = self.conv7.kernel_size[0] * self.conv7.kernel_size[1] * self.conv1.kernel_size[2]* self.conv7.out_channels
        self.conv7.weight.data.normal_(0, math.sqrt(2. / n))


        nn.init.xavier_uniform(self.fc1.weight)
        nn.init.xavier_uniform(self.fc2.weight)
        nn.init.xavier_uniform(self.fc3.weight)
        nn.init.xavier_uniform(self.fc4.weight)

        self.batch1.weight.data.fill_(1)
        self.batch1.bias.data.zero_()
        self.batch2.weight.data.fill_(1)
        self.batch2.bias.data.zero_()
        self.batch3.weight.data.fill_(1)
        self.batch3.bias.data.zero_()
        self.batch5.weight.data.fill_(1)
        self.batch5.bias.data.zero_()
        self.batch4.weight.data.fill_(1)
        self.batch4.bias.data.zero_()
        self.batch6.weight.data.fill_(1)
        self.batch6.bias.data.zero_()
        self.batch7.weight.data.fill_(1)
        self.batch7.bias.data.zero_()
        self.batch8.weight.data.fill_(1)
        self.batch8.bias.data.zero_()
        self.batch9.weight.data.fill_(1)
        self.batch9.bias.data.zero_()
        self.batch10.weight.data.fill_(1)
        self.batch10.bias.data.zero_()


    def forward(self, x):
        flats = []
        _,n,_,_,_,_ = x.shape
        for i in range(n):
            y = x[:,i,:,:,:,:]
            print(y.shape)
            y = F.relu(self.batch1(self.conv1(y)))
            y = self.pool(y)
            y = F.relu(self.batch2(self.conv2(y)))
            y = self.pool2(y)
            y = F.relu(self.batch3(self.conv3(y)))
            y = self.pool(y)
            y = F.relu(self.batch4(self.conv4(y)))
            y = self.pool1(y)
            y = F.relu(self.batch5(self.conv5(y)))
            y = self.pool1(y)
            y = F.relu(self.batch6(self.conv6(y)))
            y = self.pool1(y)
            y = F.relu(self.batch7(self.conv7(y)))
            y = self.pool1(y)
            y = torch.flatten(y, 1)
            flats.append(y)
        x = torch.cat(flats, dim=1)
        x = self.drop(x)
        x = F.relu(self.batch8(self.fc1(x)))
        x = self.drop(x)
        x = F.relu(self.batch9(self.fc2(x)))
        x = self.drop(x)
        x = F.relu(self.batch10(self.fc3(x)))
        x = self.drop(x)
        x = self.fc4(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
import os
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, entries, dtime, goal_coord, data, transform=None, target_transform=None):
        self.entries = entries
        self.transform = transform
        self.target_transform = target_transform
        self.dtime = dtime
        self.goal_coord = goal_coord
        self.data = data


    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        inp,lab = input_label(self.entries,self.dtime,idx, self.goal_coord,self.data)
        inp = np.pad(inp,((0,0),(0,0),(0,0),(2,2),(0,0)),'wrap')
        return inp, lab

In [ ]:
trainset = CustomDataset(training_data,3,coords_destination,data)
batch_size = 8
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

In [ ]:
x = trainset.__getitem__(0)[0]
print(x.shape)

(3, 3, 17, 148, 73)


In [ ]:
net = Net()

C:\Users\monce\AppData\Local\Temp\ipykernel_10032\4173427636.py:58: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.fc1.weight)
C:\Users\monce\AppData\Local\Temp\ipykernel_10032\4173427636.py:59: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.fc2.weight)
C:\Users\monce\AppData\Local\Temp\ipykernel_10032\4173427636.py:60: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.fc3.weight)
C:\Users\monce\AppData\Local\Temp\ipykernel_10032\4173427636.py:61: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.fc4.weight)


In [ ]:
x = torch.tensor([trainset.__getitem__(100)[0]],dtype=torch.float32)
net.eval()
output = net(x)
print(output)

torch.Size([1, 3, 17, 144, 73])


torch.Size([1, 3, 17, 144, 73])
torch.Size([1, 3, 17, 144, 73])
tensor([[0.4859, 0.3866, 0.1274]], grad_fn=<SoftmaxBackward0>)


In [ ]:
criterion = nn.MultiMarginLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)


In [ ]:
a,b = next(iter(trainloader))
print(a.shape)
net.eval()
output = net(a.float())
print(output)

torch.Size([8, 3, 3, 17, 144, 73])
torch.Size([8, 3, 17, 144, 73])
torch.Size([8, 3, 17, 144, 73])
torch.Size([8, 3, 17, 144, 73])
tensor([[0.4511, 0.4075, 0.1413],
        [0.4617, 0.3994, 0.1389],
        [0.4750, 0.4083, 0.1167],
        [0.4956, 0.3746, 0.1298],
        [0.4809, 0.3797, 0.1393],
        [0.4833, 0.3862, 0.1305],
        [0.4453, 0.4208, 0.1339],
        [0.4599, 0.3883, 0.1518]], grad_fn=<SoftmaxBackward0>)


In [ ]:
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    # exp_lr_scheduler.step()
    for i, batch in enumerate(trainloader, 0):
        print(i)
        inputs, labels = batch
        inputs = inputs.float()
        labels = labels
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[1,   100] loss: 0.615
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
[1,   200] loss: 0.599
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images
        labels = labels
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

NameError: name 'testloader' is not defined